# Music Millenium Classification

I will build Logistic Regression and K - Nearest Neighbour models to predict which **century** a piece of music was released.  I am using the "YearPredictionMSD Data Set"
based on the Million Song Dataset. The data is available to download from the UCI 
Machine Learning Repository:

- https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd

As this is not as challenging a dataset/project I will make it more interesting by implementing my Logistic Regression models using only numpy and train it using stochastic gradient descent from scratch. I will similarly implement my K-NN model from scratch!

In [0]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import time

In [0]:
load_from_drive = False

if not load_from_drive:
  csv_path = "http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip"
else:
  from google.colab import drive
  drive.mount('/content/gdrive')
  csv_path = '/content/drive/My Drive/YearPredictionMSD.txt.zip' 

t_label = ["year"]
x_labels = ["var%d" % i for i in range(1, 91)]
df = pandas.read_csv(csv_path, names=t_label + x_labels)

In [0]:
df

,year,var1,var2,var3,var4,var5,var6,var7,var8,var9,var10,var11,var12,var13,var14,var15,var16,var17,var18,var19,var20,var21,var22,var23,var24,var25,var26,var27,var28,var29,var30,var31,var32,var33,var34,var35,var36,var37,var38,var39,...,var51,var52,var53,var54,var55,var56,var57,var58,var59,var60,var61,var62,var63,var64,var65,var66,var67,var68,var69,var70,var71,var72,var73,var74,var75,var76,var77,var78,var79,var80,var81,var82,var83,var84,var85,var86,var87,var88,var89,var90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,-2.46783,3.32136,-2.31521,10.20556,611.10913,951.08960,698.11428,408.98485,383.70912,326.51512,238.11327,251.42414,187.17351,100.42652,179.19498,-8.41558,-317.87038,95.86266,48.10259,-95.66303,-18.06215,1.96984,34.42438,11.72670,1.36790,7.79444,-0.36994,-133.67852,-83.26165,-37.29765,...,-25.38187,-3.90772,13.29258,41.55060,-7.26272,-21.00863,105.50848,64.29856,26.08481,-44.59110,-8.30657,7.93706,-10.73660,-95.44766,-82.03307,-35.59194,4.69525,70.95626,28.09139,6.02015,-37.13767,-41.12450,-8.40816,7.19877,-8.60176,-5.90857,-12.32437,14.68734,-54.32125,40.14786,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,2.21920,0.34006,44.38997,2056.93836,605.40696,457.41175,777.15347,415.64880,746.47775,366.45320,317.82946,273.07917,141.75921,317.35269,19.48271,-65.25496,162.75145,135.00765,-96.28436,-86.87955,17.38087,45.90742,32.49908,-32.85429,45.10830,26.84939,-302.57328,-41.71932,-138.85034,...,28.55107,1.52298,70.99515,-43.63073,-42.55014,129.82848,79.95420,-87.14554,-45.75446,-65.82100,-43.90031,-19.45705,12.59163,-407.64130,42.91189,12.15850,-88.37882,42.25246,46.49209,-30.17747,45.98495,130.47892,13.88281,-4.00055,17.85965,-18.32138,-87.99109,14.37524,-22.70119,-58.81266,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,2.73553,0.82804,7.46586,699.54544,1016.00954,594.06748,355.73663,507.39931,387.69910,287.15347,112.37152,161.68928,144.14353,199.29693,-4.24359,-297.00587,-148.36392,-7.94726,-18.71630,12.77542,-25.37725,9.71410,0.13843,26.79723,6.30760,28.70107,-74.89005,-289.19553,-166.26089,...,18.50939,16.97216,24.26629,-10.50788,-8.68412,54.75759,194.74034,7.95966,-18.22685,0.06463,-2.63069,26.02561,1.75729,-262.36917,-233.60089,-2.50502,-12.14279,81.37617,2.07554,-1.82381,183.65292,22.64797,-39.98887,43.37381,-31.56737,-4.88840,-36.53213,-23.94662,-84.19275,66.00518,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,6.63016,-3.35142,37.64085,2174.08189,697.43346,459.24587,742.78961,229.30783,387.89697,249.06662,245.89870,176.20527,98.82222,150.97286,78.49057,-62.00282,43.49659,-96.42719,-108.96608,14.22854,14.54178,-23.55608,-39.36953,-43.59209,20.83714,35.63919,-181.34947,-93.66614,-90.55616,...,4.56917,-37.32280,4.15159,12.24315,35.02697,-178.89573,82.46573,-20.49425,101.78577,-19.77808,-21.52657,3.36303,-11.63176,51.55411,-50.57576,-28.14755,-83.15795,-7.35260,-22.11505,1.18279,-122.70467,150.57360,24.37468,41.19821,-37.04318,-28.72986,162.19614,22.18309,-8.63509,85.23416,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,1.60923,2.19223,47.32082,894.28471,809.86615,318.78559,435.04497,341.61467,334.30734,322.99589,190.61921,235.84715,96.89517,210.58870,5.60463,-199.63958,204.85812,-77.17695,-65.79741,-6.95097,-12.15262,-3.85410,20.68990,-20.30480,37.15045,11.20673,-124.09519,-295.98542,-33.31169,...,45.25506,10.42226,27.88782,-17.12676,-31.54772,-76.86293,41.17343,-138.32535,-53.96905,-21.30266,-24.87362,-2.46595,-4.0500

To set up my data for classification, I'll use the "year" field to represent
whether a song was released in the 20-th century. In this case, `df["year"]` will be 1 if
the year of release is after 2000, and 0 otherwise.

In [0]:
df["year"] = df["year"].map(lambda x: int(x > 2000))

When I split my data into train/test I need to
avoid the "producer effect". That is, I want to make sure that no song from a single artist ends up in both the training and test set.

Having the same artist in both the training set and test set will mean the 
model will classify that artist's songs very easily. The test set is meant to 
give an unbiased estimate of the model's performance on unseen data. It allows
us to understand how well the model will generalize to out of sample data. But 
having songs from the same artist in both training and testing sets will give 
us a biased test accuracy.

In [0]:
df_train = df[:463715]
df_test = df[463715:]

# convert to numpy
train_xs = df_train[x_labels].to_numpy()
train_ts = df_train[t_label].to_numpy()
test_xs = df_test[x_labels].to_numpy()
test_ts = df_test[t_label].to_numpy()

It can be beneficial to **normalize** the columns, so that each feature
has the *same* mean and standard deviation.

In [0]:
feature_means = df_train.mean()[1:].to_numpy() # the [1:] removes the mean of the "year" field
feature_stds  = df_train.std()[1:].to_numpy()

train_norm_xs = (train_xs - feature_means) / feature_stds
test_norm_xs = (test_xs - feature_means) / feature_stds

Note that I normalized the test set using the *training data means and standard deviations*. This is because if I compute/transform any features using any information from the test set I will be victim to data leakage.

In [0]:
# shuffle the training set
reindex = np.random.permutation(len(train_xs))
train_xs = train_xs[reindex]
train_norm_xs = train_norm_xs[reindex]
train_ts = train_ts[reindex]

# using the first 50000 elements of `train_xs` as the validation set
train_xs, val_xs           = train_xs[50000:], train_xs[:50000]
train_norm_xs, val_norm_xs = train_norm_xs[50000:], train_norm_xs[:50000]
train_ts, val_ts           = train_ts[50000:], train_ts[:50000]

In [0]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))
    
def cross_entropy(t, y):
  return -t * np.log(y) - (1 - t) * np.log(1 - y)

def cost(y, t):
  return np.mean(cross_entropy(t, y))

def get_accuracy(y, t):
  acc = 0
  N = 0
  for i in range(len(y)):
    N += 1
    if (y[i] >= 0.5 and t[i] == 1) or (y[i] < 0.5 and t[i] == 0):
      acc += 1
  return acc / N

In [0]:
def pred(w, b, X):
  """
  Returns the prediction `y` of the target based on the weights `w` and scalar bias `b`.

  Preconditions: np.shape(w) == (90,)
                 type(b) == float
                 np.shape(X) = (N, 90) for some N
  """
  return sigmoid(np.matmul(X,w)+b)

In [0]:
def derivative_cost(X, y, t):
  """
  Returns a tuple containing the gradients dEdw and dEdb.

  Precondition: np.shape(X) == (N, 90) for some N
                np.shape(y) == (N,)
                np.shape(t) == (N,)

  Postcondition: np.shape(dEdw) = (90,)
           type(dEdb) = float
  """
  dLdz=(1/t.shape[0])*(y-t)
  return np.matmul(X.T, dLdz), np.mean(y-t)

To check that my derivative is implemented correctly I can verify the results using the finite difference rule. In 1D, the
finite difference rule says that for small $h$, I should have

$$\frac{f(x+h) - f(x)}{h} \approx f'(x)$$


In [0]:
def verify_derivative_of_bias(w, b, X, t, h):
  finite_difference = (cost(pred(w, b + h, X), t) - cost(pred(w, b, X), t)) / h
  return np.abs(derivative_cost(X, pred(w, b, X), t)[1]-finite_difference)<0.000001

verify_derivative_of_bias(np.zeros(90), 1, np.ones([2, 90]), np.array([1,1]),0.000001)

True

In [0]:
h = 0.000001
H = np.zeros(90)
H[0] = h
def verify_derivative_of_weights(w, b, X, t, h):
  finite_difference = (cost(pred(w+h, b, X), t) - cost(pred(w, b, X), t)) / h
  return np.abs(derivative_cost(X, pred(w, b, X), t)[0]-finite_difference)[0]<0.000001

verify_derivative_of_weights(np.zeros(90), 1, np.ones([2, 90]), np.array([1,1]),H)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


True

Now that my gradient function works, I will run stochastic gradient descent to train my weights and biases.

In [0]:
def run_gradient_descent(w0, b0, alpha=0.1, batch_size=100, max_iters=100):
  """Return the values of (w, b) after running gradient descent for max_iters.

    - alpha as the learning rate
    - (w0, b0) as the initial values of (w, b)

  Precondition: np.shape(w0) == (90,)
                type(b0) == float
 
  Postcondition: np.shape(w) == (90,)
                 type(b) == float
  """
  w = w0
  b = b0
  iter = 0
  global train_norm_xs
  global train_ts
  global val_norm_xs
  global val_ts
  while iter < max_iters:
    # shuffle the training set
    reindex = np.random.permutation(len(train_norm_xs))
    train_norm_xs = train_norm_xs[reindex]
    train_ts = train_ts[reindex]
    for i in range(0, len(train_norm_xs), batch_size): # iterate over each minibatch
      # current minibatch
      X = train_norm_xs[i:(i + batch_size)]
      t = train_ts[i:(i + batch_size), 0]

      # if len(train_norm_xs) does not divide batch_size evenly, I will skip over
      # the "last" minibatch
      if np.shape(X)[0] != batch_size:
        continue

      # compute the prediction
      y = pred(w,b,X)

      # update w and b
      dw, db = derivative_cost(X,y,t)
      w -= alpha*dw
      b -= alpha*db

      # increment the iteration count
      iter += 1

      # compute and plot the *validation* loss and accuracy
      if (iter % 10 == 0):
        X_val = val_norm_xs[i:(i + batch_size)]
        t_val = val_ts[i:(i + batch_size), 0]
        train_cost = cost(y,t)
        val_y = pred(w,b,X_val)
        val_cost = cost(val_y,t_val)
        val_acc = get_accuracy(val_y,t_val)
        print("Iter %d. [Val Acc %.0f%%, Loss %f] [Train Loss %f]" % (
               iter, val_acc * 100, val_cost, train_cost))

      if iter >= max_iters:
        break
  return w,b

In [0]:
w0 = np.zeros(90)
b0 = 0.
print("Alpha = 10e-18: ")
t0 = time.time()
w,b = run_gradient_descent(w0,b0,alpha=10e-18)
print("total time: {}s".format(time.time() - t0))

print("\nAlpha = 1: ")
t0 = time.time()
w,b = run_gradient_descent(w0,b0,alpha=1)
print("total time: {}s".format(time.time() - t0))
#if alpha is really small we see the training loss reducing very very slowly, but still reducing
#if alpha is really big we see the training loss bouncing lower then higher then lower erratically, 
#without showing any signs of convergence. 

Alpha = 10e-18: 
Iter 10. [Val Acc 60%, Loss 0.693147] [Train Loss 0.693147]
Iter 20. [Val Acc 55%, Loss 0.693147] [Train Loss 0.693147]
Iter 30. [Val Acc 54%, Loss 0.693147] [Train Loss 0.693147]
Iter 40. [Val Acc 52%, Loss 0.693147] [Train Loss 0.693147]
Iter 50. [Val Acc 49%, Loss 0.693147] [Train Loss 0.693147]
Iter 60. [Val Acc 67%, Loss 0.693147] [Train Loss 0.693147]
Iter 70. [Val Acc 58%, Loss 0.693147] [Train Loss 0.693147]
Iter 80. [Val Acc 58%, Loss 0.693147] [Train Loss 0.693147]
Iter 90. [Val Acc 68%, Loss 0.693147] [Train Loss 0.693147]
Iter 100. [Val Acc 68%, Loss 0.693147] [Train Loss 0.693147]
total time: 0.11827945709228516s

Alpha = 1: 
Iter 10. [Val Acc 62%, Loss 0.710471] [Train Loss 0.695492]
Iter 20. [Val Acc 60%, Loss 0.817926] [Train Loss 0.710880]
Iter 30. [Val Acc 64%, Loss 0.736807] [Train Loss 0.688956]
Iter 40. [Val Acc 68%, Loss 0.586676] [Train Loss 0.580556]
Iter 50. [Val Acc 71%, Loss 0.650352] [Train Loss 0.697245]
Iter 60. [Val Acc 80%, Loss 0.447095

In [0]:
# I chose the learning rate and batch size by tuning them on the validation set 
# unit I started seeing a good convergence of validation loss as well as an 
# increase in the validation accuracy.
w0 = np.zeros(90)
b0 = 0.
w,b = run_gradient_descent(w0,b0,alpha=.1, batch_size=200, max_iters=150)

Iter 10. [Val Acc 64%, Loss 0.667504] [Train Loss 0.659990]
Iter 20. [Val Acc 68%, Loss 0.619962] [Train Loss 0.625714]
Iter 30. [Val Acc 76%, Loss 0.595841] [Train Loss 0.612246]
Iter 40. [Val Acc 76%, Loss 0.558212] [Train Loss 0.611428]
Iter 50. [Val Acc 72%, Loss 0.601272] [Train Loss 0.583014]
Iter 60. [Val Acc 70%, Loss 0.605956] [Train Loss 0.607637]
Iter 70. [Val Acc 72%, Loss 0.614288] [Train Loss 0.578122]
Iter 80. [Val Acc 68%, Loss 0.622201] [Train Loss 0.558161]
Iter 90. [Val Acc 71%, Loss 0.605193] [Train Loss 0.583321]
Iter 100. [Val Acc 76%, Loss 0.567586] [Train Loss 0.585605]
Iter 110. [Val Acc 69%, Loss 0.578377] [Train Loss 0.577354]
Iter 120. [Val Acc 75%, Loss 0.558678] [Train Loss 0.568463]
Iter 130. [Val Acc 74%, Loss 0.575937] [Train Loss 0.564099]
Iter 140. [Val Acc 70%, Loss 0.548982] [Train Loss 0.588063]
Iter 150. [Val Acc 78%, Loss 0.568897] [Train Loss 0.646459]


In [0]:
print('Training Accuracy: ', get_accuracy(pred(w,b,train_norm_xs),train_ts))
print('Validation Accuracy: ', get_accuracy(pred(w,b,val_norm_xs),val_ts))
print('Test Accuracy: ', get_accuracy(pred(w,b,test_norm_xs),test_ts))
"""The Test accuracy is lower than the training set and validation set accuracy
as expected. This is because the model tends to fit the nuances/noice in the
training set to a certain degree, and hence has a higher accuracy for that data
in comparison to the test set. """

Training Accuracy:  0.7216658811017246
Validation Accuracy:  0.72494
Test Accuracy:  0.7131900058105752


'The Test accuracy is lower than the training set and validation set accuracy\nas expected. This is because the model tends to fit the nuances/noice in the\ntraining set to a certain degree, and hence has a higher accuracy for that data\nin comparison to the test set.'

### Building Logistic Regression models with sklearn instead now

In [0]:
import sklearn.linear_model
model = sklearn.linear_model.LogisticRegression()
model.fit(train_norm_xs,train_ts)
print('Training Accuracy: ', model.score(train_norm_xs, train_ts))
print('Validation Accuracy: ', model.score(val_norm_xs, val_ts))
print('Test Accuracy: ', model.score(test_norm_xs, test_ts))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Accuracy:  0.7326927957652006
Validation Accuracy:  0.73454
Test Accuracy:  0.7263800116211505


## Nearest Neighbour

I will now compare the nearest neighbour model with the model I built earlier.

To make predictions for a new data point using k-nearest neighbour, I will need to:

1. Compute the distance from this new data point to every element in the training set
2. Select the top $k$ closest neighbour in the training set
3. Find the most common label among those neighbours

I'll use the validation test to select $k$. 

Since I have a fairly large data set, computing the distance between a point in the validation
set and all points in the training set will require more RAM than Google Colab provides.
To make the comptuations tractable, I will:

1. Use only a subset of the training set (only the first 100,000 elements)
2. Use only a subset of the validation set (only the first 1000 elements)
3. I will use the **cosine similarity** rather than Euclidean distance. I will also pre-scale
   each element in training set and the validation set to be a unit vector, so that computing
   the cosine similarity is equivalent to computing the dot product. ie: 
   $$cos(\theta) = \frac{v \cdot w}{||v|| ||w||}$$ But if both ||v|| and ||w|| are zero, then
   only the dot product remains.

In [0]:
def scale_data_to_unit_vector(dataset):
  norms = np.linalg.norm(dataset, axis=1)
  return (dataset.T / norms).T

xs = scale_data_to_unit_vector(train_norm_xs[:100000])

In [0]:
val_xs = scale_data_to_unit_vector(val_norm_xs[:1000])

The goal now is to compute the validation accuracy for a choice of $k$. This will
require computing the distance between each song in the training set and each
song in the validation set.

This is actually quite straightforward, and can be done using one matrix
computation operation!

In [0]:
def get_distance_between_x1_and_x2(x1, x2):
  return np.dot(x1, x2.T)
val_distances = get_distance_between_x1_and_x2(xs, val_xs)

In [0]:
def get_nearest_neighbours(i, k=10, distances=val_distances):
  """Return the indices of the top k-element of `xs` that are closests to
  element `i` of the dataset.
  """
  # sort the element of the training set by distance to the i-th element
  neighbours = sorted(enumerate(distances[:, i]),
                      key=lambda r: r[1],
                      reverse=True)
  # obtain the top k closest index and return it
  neighbour_indices = [index for (index, dist) in neighbours[:k]]
  return neighbour_indices 

def get_train_ts(indices):
  """Return the labels of the corresponding elements in the training set `xs`.
  Note that `xs` is the first 100,000 elements of `train_xs`, so I can
  simply index `train_ts`.
  """
  return train_ts[indices]

def get_label_from_K_closest_labels(labels):
  return np.bincount(labels.flatten()).argmax()

def get_prediction_for_element(i, k, distances):
  return get_label_from_K_closest_labels(get_train_ts(get_nearest_neighbours(i, k=k, distances=distances)))

def get_prediction_for_set(data_set, k, distances=val_distances):
    return [get_prediction_for_element(x, k=k, distances=distances)for x in range(len(data_set))]
print('Validation Accuracy for k=10: ', get_accuracy(get_prediction_for_set(val_xs, 10), val_ts[:1000]))

Validation Accuracy for k=10:  0.692


In [0]:
print('Validation Accuracy for k=50: ', get_accuracy(get_prediction_for_set(val_xs, 50), val_ts[:1000]))
print('Validation Accuracy for k=100: ', get_accuracy(get_prediction_for_set(val_xs, 100), val_ts[:1000]))
print('Validation Accuracy for k=1000: ', get_accuracy(get_prediction_for_set(val_xs, 1000), val_ts[:1000]))
#K=100 seems like the best model to deploy

Validation Accuracy for k=50:  0.698
Validation Accuracy for k=100:  0.704
Validation Accuracy for k=1000:  0.687


Computing the test accuracy for the best $k$ from above.
Again I am using only a sample of 1000 elements from the test set to keep the problem tractable.

In [0]:
test_xs = scale_data_to_unit_vector(test_xs[:1000])
print('Test Accuracy for k=100: ', 
      get_accuracy(get_prediction_for_set(val_xs, 100, 
      distances=get_distance_between_x1_and_x2(xs, test_xs)), test_ts[:1000]))

Test Accuracy for k=100:  0.555
